In [10]:
import numpy as np
import pandas as pd
import cobra
from cobra.io import read_sbml_model
import matplotlib.pyplot as plt
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.flux_analysis import pfba
from IPython.display import display, Markdown

In [11]:
import time
from itertools import product

from itertools import product
import time

def find_combinations(param_names, combination_size, step=0.01):
    """
    Finds combinations of values for exactly `combination_size` parameters
    where the sum of their absolute values equals 1, including `biomass` and `igg`
    and cycling through the remaining parameters.
    
    Args:
        param_names: A list of parameter names (strings).
        combination_size: The number of non-zero coefficients in each combination (including biomass and igg).
        step: The step size for iterating through values (default: 0.01).
    
    Returns:
        A list of valid combinations for the given combination size.
    """
    # Start timing the process
    start_time = time.time()

    # Prepare the value options for each parameter based on its name
    value_options_dict = {}
    
    for param in param_names:
        if param in [biomass, igg]:
            # Only non-zero values between 0 and 1 (exclude 0)
            value_range = (0.0, 1.0)
            value_options_dict[param] = [round(x * step, 2) for x in range(1, int(value_range[1] / step) + 1)]  # Skip 0
        elif param in [lactate, glutamine, glucose, ammonia]:
            # Only non-positive values between -1 and 0
            value_range = (-1.0, 0.0)
            value_options_dict[param] = [round(x * step, 2) for x in range(int(value_range[0] / step), int(value_range[1] / step) + 1)]
        else:
            # Default range for other parameters
            value_range = (-1.0, 1.0)
            value_options_dict[param] = [round(x * step, 2) for x in range(int(value_range[0] / step), int(value_range[1] / step) + 1)]
    
    # Generate all combinations for the given number of non-zero reactions
    valid_combinations = []
    
    # We must ensure that 'biomass' and 'igg' are always included in the combinations
    selected_params = [biomass, igg] + [param for param in param_names if param not in [biomass, igg]]
    
    # Loop over all parameters, but only select a subset of combination_size
    for selected_subset in product(*[value_options_dict[param] for param in selected_params[:combination_size]]):
        non_zero_count = sum(1 for x in selected_subset if x != 0)

        if non_zero_count == combination_size:
            # Check if the sum of absolute values equals 1.0
            if abs(sum(abs(x) for x in selected_subset) - 1.0) < 0.0001:  # Small tolerance for floating point errors
                valid_combinations.append(dict(zip(selected_params[:combination_size], selected_subset)))

    return valid_combinations



def viable_loopless_fba_solutions(reaction_names, positive_flux_list, negative_flux_list):
    """
    Finds viable loopless FBA solutions for any number of reactions, ensuring flux constraints.

    Args:
        reaction_names: A list of reaction names to be used in the objective function.
        positive_flux_list: A list of reactions that should have positive flux.
        negative_flux_list: A list of reactions that should have negative flux.

    Returns:
        A list of viable objective function dictionaries that meet the flux constraints.
    """
    
    viable_objectives = []
    
    # Start the timer
    start_time = time.time()

    # Iterate over the number of non-zero coefficients starting from 3
    for combination_size in range(3, len(reaction_names) + 1):
        print(f"Generating combinations of size {combination_size}...")

        # Generate combinations for the current combination size
        objective_combinations = find_combinations(reaction_names, combination_size)

        # Check each combination for feasibility
        for index, objective in enumerate(objective_combinations):
            # Print current objective for debugging/monitoring
            print(f"Checking objective with {combination_size} non-zero coefficients: {objective}")
            
            # Here, you perform the FBA and loopless solution checking
            with model:
                model.objective = objective
                add_pfba(model, objective=objective)
                solution = model.optimize()
                loopless = loopless_solution(model).fluxes
            
                # Check if all reactions in positive_flux_list have positive flux
                positive_flux_ok = all(loopless.loc[reaction] > 0 for reaction in positive_flux_list)
                
                # Check if all reactions in negative_flux_list have negative flux
                negative_flux_ok = all(loopless.loc[reaction] < 0 for reaction in negative_flux_list)
            
                if positive_flux_ok and negative_flux_ok:
                    viable_objectives.append(objective)  # Store valid objectives
                    print(f"Found viable solution: {objective}")
                    return viable_objectives  # Return immediately after finding the first feasible solution

            # Estimate and print progress every 1000 iterations or on the final iteration
            elapsed_time = time.time() - start_time
            progress = (index + 1) / len(objective_combinations)
            remaining_time = elapsed_time / progress - elapsed_time  # Estimate remaining time
            print(f"Progress: {progress * 100:.2f}% - Estimated time left: {remaining_time:.2f} seconds")

    return viable_objectives


In [12]:
model = read_sbml_model('iCHO2441_221-107_producing.xml')
model

Name,iCHO2441_221107_producing
Memory address,136df7a1430
Number of metabolites,4174
Number of reactions,6337
Number of genes,2441
Number of groups,15
Objective expression,1.0*biomass_cho_prod - 1.0*biomass_cho_prod_reverse_1b5b7
Compartments,"cytosol, lysosome, mitochondria, endoplasmicReticulum, nucleus, extracellularSpace, peroxisome, golgiApparatus, secretoryVesicle"


In [13]:
bounds_df = pd.read_csv('bounds_df.csv')

for index, row in bounds_df.iterrows():
    reaction = model.reactions.get_by_id(row['reaction'])
    reaction.lower_bound = row['lower bound']
    reaction.upper_bound = row['upper bound']

In [14]:
mismatches = []
for index, row in bounds_df.iterrows():
    reaction = model.reactions.get_by_id(row['reaction'])
    if reaction.lower_bound != row['lower bound'] or reaction.upper_bound != row['upper bound']:
        mismatches.append((row['reaction'], reaction.lower_bound, reaction.upper_bound, row['lower bound'], row['upper bound']))

# Print mismatches if any
if mismatches:
    print(f"{len(mismatches)} reactions have incorrect bounds:")
    for rxn, lb_model, ub_model, lb_csv, ub_csv in mismatches[:10]:  # Show first 10 mismatches
        print(f"{rxn}: Model({lb_model}, {ub_model}) != CSV({lb_csv}, {ub_csv})")
else:
    print("All reaction bounds were correctly updated!")

All reaction bounds were correctly updated!


In [15]:
#remove non-negative bound on lactate and ammonia exchange reactions

model.reactions.get_by_id('EX_lac_L(e)').lower_bound = -1000
model.reactions.get_by_id('EX_nh4(e)').lower_bound = -1000

In [16]:
print('the current model objective function is:',model.objective)
solution = model.optimize()

the current model objective function is: Maximize
1.0*biomass_cho_prod - 1.0*biomass_cho_prod_reverse_1b5b7


In [17]:
igg = model.reactions.get_by_id('igg_formation')
lactate = model.reactions.get_by_id('EX_lac_L(e)')
glutamine = model.reactions.get_by_id('EX_gln_L(e)')
glucose = model.reactions.get_by_id('EX_glc(e)')
ammonia = model.reactions.get_by_id('EX_nh4(e)')
biomass = model.reactions.get_by_id('biomass_cho_prod')

positive_fluxes = ['biomass_cho_prod', 'igg_formation']
negative_fluxes = ['EX_lac_L(e)', 'EX_nh4(e)']

In [ ]:
all_solutions = viable_loopless_fba_solutions([biomass, igg, glucose, glutamine, ammonia, lactate], positive_fluxes, negative_fluxes)

Generating combinations of size 3...
Checking objective with 3 non-zero coefficients: {<Reaction biomass_cho_prod at 0x136e2a08da0>: 0.01, <Reaction igg_formation at 0x136e2b5f530>: 0.01, <Reaction EX_glc(e) at 0x136e1176810>: -0.98}
Progress: 0.02% - Estimated time left: 55971.61 seconds
Checking objective with 3 non-zero coefficients: {<Reaction biomass_cho_prod at 0x136e2a08da0>: 0.01, <Reaction igg_formation at 0x136e2b5f530>: 0.02, <Reaction EX_glc(e) at 0x136e1176810>: -0.97}
